In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [105]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [106]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
ohe=OneHotEncoder()

In [107]:
data['Gender']=le.fit_transform(data['Gender'])
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


0=Female, 1=Male

In [108]:
encodedGeography=ohe.fit_transform(pd.DataFrame(data['Geography'])).toarray()

In [109]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [110]:
encodedGeography=pd.DataFrame(encodedGeography,columns=ohe.get_feature_names_out(['Geography']))
encodedGeography.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [111]:
data=pd.concat([data,encodedGeography],axis=1)
data.drop(['RowNumber','CustomerId','Surname','Geography'],axis=1,inplace=True)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [112]:
data.shape

(10000, 13)

---
<br>

# ANN start

In [113]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [114]:
x=data.drop('Exited',axis=1)
y=data.Exited

In [115]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=19)

In [116]:
def tuner(neurons=64,hiddenLayer=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(x.shape[1],)))

    for _ in range(hiddenLayer-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    
    opt=tf.keras.optimizers.Adam(learning_rate=0.01)
    loss=tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

    return model

In [124]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import  GridSearchCV

es=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
model=tuner(64,4)

d:\ML\11\env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [125]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),callbacks=[es],verbose=1)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6732 - loss: 494.2016 - val_accuracy: 0.7905 - val_loss: 1.3680
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6974 - loss: 1.3645 - val_accuracy: 0.7860 - val_loss: 0.5407
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7387 - loss: 0.7037 - val_accuracy: 0.7965 - val_loss: 0.5400
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7485 - loss: 0.6589 - val_accuracy: 0.7985 - val_loss: 0.5028
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7731 - loss: 0.5675 - val_accuracy: 0.7985 - val_loss: 0.5356
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7745 - loss: 0.6549 - val_accuracy: 0.7985 - val_loss: 12.8926
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7367 - loss: 4.0641 - val_accuracy: 0.7985 - val_loss: 0.5013
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7480 - loss: 0.6796 - val_a

In [128]:
from sklearn.metrics import  classification_report

In [142]:
preds=model.predict(x_test)
preds[:,0]

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([0.22576758, 0.22576758, 0.22576758, ..., 0.22576758, 0.17301424,
       0.17798018], dtype=float32)

In [140]:
y_test

1921    1
530     0
3931    0
8585    1
1937    0
       ..
2444    0
6240    0
3471    0
6488    0
1068    1
Name: Exited, Length: 2000, dtype: int64